In [1]:
import os
import pandas as pd
import re
import numpy as np

import collections
from operator import itemgetter
import pprint as pp
import re
from nltk.corpus import stopwords
import nltk.data
import time
import proj_base
#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [16]:
data = proj_base.getStandardData(numFiles=3)
data.shape

(302, 13)

In [3]:
data.head()

,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008\r",5.0,5.0,NaN,selizabethm\r,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008\r",5.0,5.0,NaN,IndieLady\r,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008\r",3.0,NaN,4,Hilobb\r,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008\r",5.0,5.0,5,Chianti_girl24\r,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008\r",NaN,NaN,NaN,hothearted\r,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [4]:
#creating the vocab of all the words

def lintWord(w):
    regex = re.compile(r'(,|\(|\)|!|:|$|\.)')
    w = re.sub(regex, '', w)
    return w


allWords = ""
for r in data["Content"]:
    #add word to big content string
    allWords += r + " "
    
#split the string at spaces, keep only unique
words = set(allWords.split(" "))


vocab = list(set([lintWord(w) for w in words if not w in stopwords.words("english")]))

#n eed to remove stopwords again because some of them may have had punctuation 
# at the end and didnt get caught the first time
vocab = [w for w in vocab if not w in stopwords.words("english") and len(w) > 2]

#vocab
print(len(vocab))

In [5]:

"""Algorithm: Aspect Segmentation Algorithm
Input: A collection of reviews {푑1, 푑2, . . . , 푑∣퐷∣)}, set of
aspect keywords {푇1, 푇2, . . . , 푇푘}, vocabulary V, selection
threshold p and iteration step limit I.
Output: Reviews split into sentences with aspect assignments.
Step 0: Split all reviews into sentences, 푋 =
{푥1, 푥2, . . . , 푥푀};
Step 1: Match the aspect keywords in each sentence
of X and record the matching hits for each aspect i in
퐶표푢푛푡(푖);
Step 2: Assign the sentence an aspect label by 푎푖 =
푎푟푔푚푎푥푖 퐶표푢푛푡(푖). If there is a tie, assign the sentence
with multiple aspects.
Step 3: Calculate chi^2 measure of each word (in V);
Step 4: Rank the words under each aspect with respect
to their chi^2value and join the top p words for each aspect
into their corresponding aspect keyword list 푇푖;
Step 5: If the aspect keyword list is unchanged or iteration
exceeds I, go to Step 6, else go to Step 1;
Step 6: Output the annotated sentences with aspect
assignments."""

'Algorithm: Aspect Segmentation Algorithm\nInput: A collection of reviews {\xed\x91\x911, \xed\x91\x912, . . . , \xed\x91\x91\xe2\x88\xa3\xed\x90\xb7\xe2\x88\xa3)}, set of\naspect keywords {\xed\x91\x871, \xed\x91\x872, . . . , \xed\x91\x87\xed\x91\x98}, vocabulary V, selection\nthreshold p and iteration step limit I.\nOutput: Reviews split into sentences with aspect assignments.\nStep 0: Split all reviews into sentences, \xed\x91\x8b =\n{\xed\x91\xa51, \xed\x91\xa52, . . . , \xed\x91\xa5\xed\x91\x80};\nStep 1: Match the aspect keywords in each sentence\nof X and record the matching hits for each aspect i in\n\xed\x90\xb6\xed\x91\x9c\xed\x91\xa2\xed\x91\x9b\xed\x91\xa1(\xed\x91\x96);\nStep 2: Assign the sentence an aspect label by \xed\x91\x8e\xed\x91\x96 =\n\xed\x91\x8e\xed\x91\x9f\xed\x91\x94\xed\x91\x9a\xed\x91\x8e\xed\x91\xa5\xed\x91\x96 \xed\x90\xb6\xed\x91\x9c\xed\x91\xa2\xed\x91\x9b\xed\x91\xa1(\xed\x91\x96). If there is a tie, assign the sentence\nwith multiple aspects.\nStep 3

In [38]:
# alternate method to get keywords. LARA method based on Chi-square is basically same, but much better
def aspectSegmentationBayes(reviews, seeds, freq_threshold = .5, prob_threshold = 0.2, words_per_iter = 4, iters = 3):

    #break down reviews into sentences and break down each sentence into words using tokenizer and remove stopwords
    # returns list where each item is the list of words in that sentence
    sentence_words = []
    for review in reviews:
        review = review.decode('utf-8')
        sentences = nltk.tokenize.sent_tokenize(review)
        for sentence in sentences:
            sentence_words.append([x.lower() for x in nltk.tokenize.word_tokenize(sentence) if x not in stopwords.words('english') and len(x) > 2])    
    

    # find Probability(sentence(S) has aspect(A) GIVEN S has word(W)) = count(S that have A and have W) / count(S that have W)

    for i in range(iters):
        
        sents_with_word_asp = {}
        sents_with_word = {}
        sents_with_aspect = {}
        prob_asp_given_word = {}

        # calculates counts of (S that have W) and (S that have A and W)
        for sentence in sentence_words:
            for word in sentence:
                sents_with_word[word] = sents_with_word.get(word,0) + 1
                for aspect, aspect_words in seeds.items():
                    for aspect_word in aspect_words:
                        if aspect_word in sentence:
                            sents_with_word_asp[(word,aspect)] = sents_with_word_asp.get((word, aspect), 0) + 1
                            sents_with_aspect[aspect] = sents_with_aspect.get(aspect,0) + 1
                            break

        for (word, aspect), count in sents_with_word_asp.items():
            #susceptible to low frequencies. hence freq_threshold
            #freq_threshold ensures that count(S with  W) is atleast x% of count(S)
            if sents_with_word[word] > (freq_threshold/100.0)*len(sentence_words):
                prob_asp_given_word[(word,aspect)] = count/float(sents_with_word[word])

        prob_asp_given_word_sorted = sorted(prob_asp_given_word.items(), key=itemgetter(1),reverse=True)
        
        for aspect, word_list in seeds.items():
            count = 0
            for item in prob_asp_given_word_sorted:
                #item is of the form ((word,aspect),probability)
                if item[0][1] == aspect:
                    if item[0][0] not in word_list:
                        if count <= words_per_iter:
                            if item[1] >= prob_threshold:
                                seeds[aspect].append(item[0][0])
                                count += 1
                            else:
                                #because sorted, the others can't have higher probability
                                break
                        else:
                            # because limiit of words per aspect in this iteration has been reached
                            break

    return seeds
start = time.time()
#sentencesWLabels = aspectSegmentation(data["Content"], seeds, vocab)
end = time.time()
sds = aspectSegmentationBayes(data["Content"], seeds)

In [40]:
#seeds will have changed, whats differet
seeds

{'Business service': ['business',
  'center',
  'computer',
  'internet',
  u'access',
  u'work',
  u'free',
  u'used',
  u'etc',
  u'shuttle',
  u'travel',
  u'centre',
  u'worked',
  u'complimentary',
  u'hour',
  u'blocks',
  u'wine',
  u'within',
  u'per'],
 'Check in / front desk': ['stuff',
  'check',
  'help',
  'reservation',
  u'hot',
  u'website',
  u'early',
  u'said',
  u'clerk',
  u'check-in',
  u'anything',
  u'arrived',
  u'desk',
  u'checked',
  u'front',
  u'late',
  u'told',
  u'rude',
  u'different'],
 'Cleanliness': ['clean',
  'dirty',
  'maintain',
  'smell',
  u'expected',
  u'bathroom',
  u'nicely',
  u'comfortable',
  u'quiet',
  u'spacious',
  u'decorated',
  u'plenty',
  u'pillows',
  u'bed',
  u'sheets',
  u'king',
  u'size',
  u'beds',
  u'tub'],
 'Location': ['location',
  'traffic',
  'minute',
  'restaurant',
  u'bar',
  u'convenient',
  u'perfect',
  u'excellent',
  u'fabulous',
  u'restaurants',
  u'although',
  u'good',
  u'great',
  u'grocery',
  u'v

In [ ]:
def seedInReview(review, seeds):
    for s in seeds:
        #print(s)
        if s in review:
            return True
    return False


reviewsWithSeeds = {}
for s in seeds:
    reviewsWithSeeds[s] = sum(data["Content"].apply(seedInReview, args = [seeds[s]]))
reviewsWithSeeds

In [ ]:
def getContentLen(rev):
    return len(rev["Content"])

data.apply(getContentLen, axis=1)

In [43]:
from nltk.tokenize import RegexpTokenizer

In [107]:
data = proj_base.getStandardData(numFiles=3)

In [111]:
aspect = "Value"

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
#okenizer = RegexpTokenizer(r'\w+')
def filterReviewSentencesByWords(rev, words):
    #tokenize review into sentences
    sentences = tokenizer.tokenize(rev["Content"])
    
    #print(sentences)
    aspSentences = []
    for s in sentences:
        wordlist = re.sub("[^a-zA-Z]"," ", s).split()
        intersect = set(wordlist).intersection(words)
        #print(s)
        if len(intersect) != 0:
            aspSentences.append(s)

    #print(aspSentences)
    if len(aspSentences) == 0:
        #so no sentences contain the rating? what do we do ehre
        
        pass
    
    rev["aspectSentences"] = aspSentences

    return rev

def getAspectSentencesForReview(data, seedWords):
    #we have our seeds, now let's only keep the sentences which are relevant to that aspect in each review 
    
    data = data.apply(filterReviewSentencesByWords, axis=1, args=(seedWords,))
    return data

def getTrainingData(data):
    includeColumns = ["aspectRating", "aspectSentences", "Content", "Overall", ]
    data = getAspectSentencesForReview(data, seeds[aspect])
    data["aspectRating"] = data[aspect]
    return data[(pd.notnull(data[aspect])) & (data["aspectSentences"].apply(len) > 0)][includeColumns]
    
data = getAspectSentencesForReview(data, seeds[aspect])

In [112]:
print(data.shape)
trainingData = getTrainingData(data)
print(data.shape, trainingData.shape)

(302, 14)
((302, 14), (147, 4))


In [113]:
#data[(pd.notnull(data[aspect])) & (data["aspectSentences"].apply(len) > 0)]
trainingData.head()

,aspectRating,aspectSentences,Content,Overall
0,4.0,[from the goldfish in the room (which my daugh...,wonderful time- even with the snow! what a gre...,5.0
2,4.0,"[nice hotel, expensive parking we got a good d...","nice hotel, expensive parking we got a good de...",4.0
3,5.0,[valet parking is the only way to go.],fabulous hotel location and service are great....,5.0
5,2.0,[well to start off i booked a suite and paid e...,"ok, but nothing special for the $$ they charge...",2.0
6,2.0,[didn't provide much guidance on special reque...,nice rooms but not a 4* experience the hotel m...,3.0


In [100]:
(pd.notnull(data[aspect])) & (data["aspectSentences"].apply(len) > 0)

0       True
1      False
2       True
3       True
4      False
5       True
6       True
7       True
8      False
9      False
10      True
11      True
12      True
13      True
14     False
15      True
16      True
17      True
18     False
19      True
20     False
21      True
22      True
23     False
24     False
25      True
26     False
27      True
28     False
29     False
       ...  
272    False
273     True
274    False
275    False
276    False
277     True
278     True
279     True
280    False
281    False
282     True
283    False
284     True
285     True
286    False
287    False
288    False
289     True
290     True
291    False
292    False
293     True
294     True
295     True
296    False
297    False
298    False
299    False
300    False
301    False
dtype: bool

In [99]:
#data["aspectSentences"].apply(len)
print(data.shape)
print(len(data[(pd.notnull(data[aspect])) & (data["aspectSentences"].apply(len) > 0)]))


(302, 14)
147


In [32]:
print(data.loc[6]["aspectSentences"])

["in comparison to the w, i spent about $45 more per night but had a larger (square footage) room with a great soaking tub (with whirlpool jets) and a nice shower.before my stay, i had the hotel arrange a car service, the price $53 with tip was reasonable and the driver was waiting for me on arrival.checkin was easy but the downside was that with a room that was picked for its 2 person jacuzi tub, i found no bath accessories (salts, bubble bath etc...) and didn't get any during the stay."]
